In [ ]:
# base_pkl = '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M10_Sk0/epistemic_uncertainty_data_collection/data.dset_type-2ms/stats_training_disentangle_2406_D25-M3-S0-L8_4.pkl'

In [ ]:
base_noise_level = '0.05'
different_noise_levels = {
    '0.2': '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M20_Sk0/epistemic_uncertainty_data_collection/stats_training_disentangle_2410_D25-M3-S0-L8_4.pkl',
    '0.05':'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M20_Sk0/epistemic_uncertainty_data_collection/stats_training_disentangle_2410_D25-M3-S0-L8_5.pkl',
    '0.5': '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M20_Sk0/epistemic_uncertainty_data_collection/stats_training_disentangle_2410_D25-M3-S0-L8_6.pkl',
    '0.8': '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M20_Sk0/epistemic_uncertainty_data_collection/stats_training_disentangle_2410_D25-M3-S0-L8_7.pkl',
    '1.0': '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M20_Sk0/epistemic_uncertainty_data_collection/stats_training_disentangle_2410_D25-M3-S0-L8_8.pkl'
}
# different_noise_levels = {
#     '0.2': '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M10_Sk0/epistemic_uncertainty_data_collection/stats_training_disentangle_2410_D25-M3-S0-L8_4.pkl',
#     '0.05':'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M10_Sk0/epistemic_uncertainty_data_collection/stats_training_disentangle_2410_D25-M3-S0-L8_5.pkl',
#     '0.5': '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M10_Sk0/epistemic_uncertainty_data_collection/stats_training_disentangle_2410_D25-M3-S0-L8_6.pkl',
#     '0.8': '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M10_Sk0/epistemic_uncertainty_data_collection/stats_training_disentangle_2410_D25-M3-S0-L8_7.pkl',
#     '1.0': '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M10_Sk0/epistemic_uncertainty_data_collection/stats_training_disentangle_2410_D25-M3-S0-L8_8.pkl'
# }


In [ ]:
# import matplotlib.pyplot as plt

# ch_idx = 0
# # img_idx = np.random.randint(0, 2000)
# _,ax = plt.subplots(figsize=(9,12),ncols=3,nrows=4)
# for row_idx, key in enumerate(different_noise_levels.keys()):
#     ax[row_idx,0].imshow(data_dict[key]['pred'][0][img_idx][0])
#     ax[row_idx,1].imshow(data_dict[key]['pred'][1][img_idx][0])
#     ax[row_idx,2].imshow(data_dict[key]['pred'][2][img_idx][0])
#     ax[row_idx,0].set_ylabel(f'{key}')

In [ ]:
import pickle
data_dict = {}
for k, v in different_noise_levels.items():
    with open(v, 'rb') as f:
        data_dict[k] = pickle.load(f)

### Compute Epistemic Uncertainty

In [ ]:
for conf_key in data_dict.keys():
    data_dict[conf_key]['std'] ={}
    for ch_idx in data_dict[conf_key]['pred'].keys():
        data_dict[conf_key]['std'][ch_idx] = []
        for img_idx in range(len(data_dict[conf_key]['pred'][ch_idx])):
            img = data_dict[conf_key]['pred'][ch_idx][img_idx]
            std = img.std(axis=0)
            data_dict[conf_key]['std'][ch_idx].append(std)


In [ ]:
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
from tqdm import tqdm
save_to_file = False

_,ax = plt.subplots(figsize=(15,4),ncols=3)
for conf_key in tqdm(data_dict.keys()):
    for ch_idx in range(3):
        std_values = data_dict[conf_key]['std'][ch_idx]
        # one value per patch
        std_values = [np.mean(x[16:-16,16:-16]) for x in std_values]
        # add linewidth=10
        sns.kdeplot(std_values,ax=ax[ch_idx], log_scale=True, label=conf_key, 
                    linewidth=3 if conf_key == base_noise_level else 1,
                    color = 'green' if conf_key == base_noise_level else None)
    #

ymin_max_dict = {}
xmin_max_dict = {}
for ch_idx in range(3):
    ax[ch_idx].grid()
    # grid background color to gray
    ax[ch_idx].set_facecolor('lightgrey')
    ymin_max_dict[ch_idx] = ax[ch_idx].get_ylim()
    xmin_max_dict[ch_idx] = ax[ch_idx].get_xlim()
    print(ch_idx, 'ymin-ymax', ax[ch_idx].get_ylim(), 'xmin-xmax', ax[ch_idx].get_xlim())

# ax[-1].legend()
# set legend title 
ax[-1].legend(title='Eval Data')

if save_to_file:
    model_id = base_pkl.split('/')[-1].replace('.pkl', '').replace('stats_training_disentangle_','')
    plot_fpath = os.path.join('/group/jug/ashesh/naturemethods/epistemic_uncertainty', f'noise_variation_{base_noise_level}_{model_id}.png')
    print(plot_fpath)
    plt.savefig(plot_fpath, dpi=200)




## Variation with amount of training data.

In [ ]:
out = 0
for ch_idx in range(3):
    out += np.array([np.mean(data_dict[key]['std'][ch_idx]) for key in ['0.05', '0.2', '0.5', '0.8', '1.0']])/3
    
plt.plot([0.05, 0.2, 0.5,0.8, 1.0], out)
# add background color to gray
plt.gca().set_facecolor('lightgrey')
plt.ylabel('Epistemic Uncertainty')
plt.xlabel('Fraction of Training Data used for Training')
plt.grid()

In [ ]:
import  torch.nn.functional as F
x = F.dropout()